In [43]:
import numpy as np
from scipy import linalg
import copy
from sympy import *
from decimal import Decimal

In [14]:
class type_of_brace:
    def __init__(self, char_type):
        self.left = '\\begin{' + char_type + 'matrix}'
        self.right = '\\end{' + char_type + 'matrix}'

Функция для печати матрицы:

In [116]:
def print_matrix(A, type_of_brace=type_of_brace('p'), precision_value=3):
    (m, n) = A.shape
    result_text = ' \n' + '  ' 
    result_text = type_of_brace.left + '\n'
    for i in range(m):
        result_text += '    '
        for j in range(n):
            if type(A[i][j]) == int or A[i][j].is_integer or type(A[i][j]) == Rational:
                result_text += (' ' + str(A[i][j]) + ' ')
            else:
                result_text += (' ' + str(np.around(float(A[i][j]), precision_value)) + ' ')
            if j != n - 1:
                result_text += '&'
        if i != m - 1:
            result_text += r'\\'
        result_text += '\n'
    result_text += '  '+ type_of_brace.right
    return result_text

Указать нужный тип для чисел и количество знаков после точки в вещественном числе, которые должны быть выведены на печать.

In [10]:
array_type = input()
precision_value = int(input())

f4
3


# Нахождение обратной матрицы

Функции для печати промежуточных вычислений и окончательного ответа соответственно:

In [36]:
def print_calculating(A, array_type, precision_value):
    n = A.shape[0]
    adjugate_array = []
    T = [type_of_brace('v'), type_of_brace('p')]
    s = 'Вычисление алгебраических дополнений матрицы $A$:\n'
    
    counter = 0
    s += '\\['
    for i in range(n):
        for j in range(n):
            I = np.ones((n, n), dtype=bool)
            I[i, :] = False
            I[:, j] = False
            I.astype(bool)
            counter += 1
            if counter == 1:
                s += '\n'
            elif counter % 2 == 0 and counter != n * n + 1:
                s += '\\;\\;\\;\\; \n'
            elif counter % 2 == 1 and counter != n * n + 1:
                s += '\\] \n\\[ \n'
            else:
                s += '\\]'
            B = A[I].copy().reshape([n - 1, n - 1])
            det_B = linalg.det(B)
            adjugate_array.append(det_B)
            s += 'A_{' + str(i + 1) + str(j + 1) + '} = (-1)^{' + str(i + 1) + '+' + str(j + 1) + '}\n  ' + print_matrix(B, T[0], precision_value) + ' = ' + str(np.around(det_B * (-1) ** (i % 2 + j % 2), decimals=precision_value)) + ';\n'
    s += '\\] \n'
    
    s += 'Получаем, что обратная матрица равна:\n'
    adjugate_matrix = np.array(adjugate_array, dtype=array_type).reshape((n, n))
    det_A = linalg.det(A)
    final_result = adjugate_matrix / det_A
    s += '\\[ \n    A^{-1} = \dfrac{1}{' + str(np.around(det_A, decimals=precision_value)) + '} ' + print_matrix(adjugate_matrix, T[1], precision_value) + ' = ' + print_matrix(final_result, T[1], precision_value) + '\\]'
    return s

Входные данные исходной матрицы $A$. На ввод подается строка вида: $a_{11} a_{12} \dots a_{1n}; a_{21} \dots a_{2n}; \dots$. В ситуации, когда обратной матрицы не существует, оно выдаст вам ошибку.

In [11]:
A = np.array(np.mat(input('A = ', )), dtype=array_type)
(m, n) = A.shape
det_A = linalg.det(A)
if n != m:
    print('Warning! Size of A is not correct!')
if det_A == 0:
    print('Warning! Determinant of A equals 0! \nOr you should change the type of elements...')

A = 1, 0; 2, 3


Вот это красиво напечатает Вам алгебраические дополнения и окончательное решение:

In [12]:
print(print_calculating(A, array_type, precision_value))

Вычисление алгебраических дополнений матрицы $A$:
\[
A_{11} = (-1)^{1+1}
  \begin{vmatrix}
     3.0 
  \end{vmatrix}
 = 3.0;
\;\;\;\; 
A_{12} = (-1)^{1+2}
  \begin{vmatrix}
     2.0 
  \end{vmatrix}
 = -2.0;
\] 
\[ 
A_{21} = (-1)^{2+1}
  \begin{vmatrix}
     0.0 
  \end{vmatrix}
 = -0.0;
\;\;\;\; 
A_{22} = (-1)^{2+2}
  \begin{vmatrix}
     1.0 
  \end{vmatrix}
 = 1.0;
\] 
Получаем, что обратная матрица равна:
\[ 
    A^{-1} = \dfrac{1}{3.0} \begin{pmatrix}
     3.0 & 2.0 \\
     0.0 & 1.0 
  \end{pmatrix}
 = \begin{pmatrix}
     1.0 & 0.667 \\
     0.0 & 0.333 
  \end{pmatrix}
\]


# Нахождение LDLT-разложения матрицы

In [89]:
n = symbols('n')

M = Matrix([[(1+n)**2, -1-n, 1-n**2, -4-5*n-n**2],
            [-1-n,1-n**2,-1-n,4+3*n-n**2],
            [1-n**2,-1-n,-13+(n-1)**2,9+n+n**2]])

M = M.subs(n, 5)
print('Исходная матрица: \n \\[ \n A=' + print_matrix(np.array(M), type_of_brace('p'), precision_value) + '\n \\]')


def ldlt_decomposition(matrix, order):
    print('Будем записывать матрицу $R^T$ в нижнем треугольнике исходной матрицы:')
    d = [0]*order
    brace_flag = True
    for j in range(0, order):
        for i in range(0, j+1):
            if (i == j):
                wi = matrix[i,i]
                for k in range(0,i):
                    wi -= d[k]*(matrix[i,k]**2)
                d[i] = sign(wi)
                matrix[i,i] = sqrt(wi*sign(wi))
            elif (i < j):
                wij = matrix[i,j]
                for k in range(0,i):
                    wij -= d[k]*(matrix[i,k])*(matrix[j,k])
                matrix[j,i] = (1 / (d[i] * matrix[i,i])) * wij
            if brace_flag:
                print('\\[')
            print(print_matrix(np.array(matrix), type_of_brace('p'), precision_value) + '\n \\rightarrow')
            if not brace_flag:
                print('\\]')
            brace_flag = not brace_flag
    print('Параллельно записываем знак $\\omega_{ii}$ в диагональную матрицу')
    print('\\[ \n D = ' + print_matrix(np.diag(d), type_of_brace('p'), precision_value) + '\n \\]')
    return matrix, d

M, D = ldlt_decomposition(M, 3)

def getRt(matrix, order):
    Rt = zeros(order)
    for i in range(0, order):
        for j in range(0, i+1):
            Rt[i,j] = matrix[i,j]
    print('\nПолучаем, что $R^T$ матрица равна:')
    print('\\[ \n R^T = ' + print_matrix(np.array(Rt), type_of_brace('p'), precision_value) + '\n \\]')
    return Rt

def solveLyb(matrix, b):
    solution = zeros(3,1)
    for i in range(3):
        curr_sol = b[i] / matrix[i,i]
        for j in range(i+1,3):
            b[j] = b[j] - b[i] * (matrix[j,i] / matrix[i,i])
        solution[i] = curr_sol
    return solution

def solveUxy(matrix, y):
    solution = zeros(3,1)
    for i in reversed(range(3)):
        curr_sol = Rational(y[i], matrix[i,i])
        for j in range(i):
            y[j] = y[j] - y[i] * Rational(matrix[j,i], matrix[i,i])
        solution[i] = curr_sol
    return solution

def Dy(D, y):
    for i in range(3):
        y[i] *= D[i]
    return y

sol = solveLyb(Rt.copy(), M[0:3, 3].copy())
sol = Dy(D, sol)
sol = solveUxy(Rt.transpose(), sol)

Rt = getRt(M, 3)
print(f'Последовательно решая системы $R^Tx_2 = (-54, -6, 39)^T$, $Dx_1 = x_2$ и $Rx = x_1$, получаем ответ $x = {tuple(sol)}.$')

Исходная матрица: 
 \[ 
 A=\begin{pmatrix}
     36 & -6 & -24 & -54 \\
     -6 & -24 & -6 & -6 \\
     -24 & -6 & 3 & 39 
  \end{pmatrix}
 \]
Будем записывать матрицу $R^T$ в нижнем треугольнике исходной матрицы:
\[
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -6 & -24 & -6 & -6 \\
     -24 & -6 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -1 & -24 & -6 & -6 \\
     -24 & -6 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\]
\[
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -1 & 5 & -6 & -6 \\
     -24 & -6 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -1 & 5 & -6 & -6 \\
     -4 & -6 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\]
\[
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -1 & 5 & -6 & -6 \\
     -4 & 2 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     6 & -6 & -24 & -54 \\
     -1 & 5 & -6 & -6 \\
     -4 & 2 & 3 & 39 
  \end{pmatrix}
 \rightarrow
\]
Параллельно записываем знак $\omega_{ii}$ в 

# Метод прогонки

In [96]:
n = symbols('n')

M = Matrix([[1, n-1, 0, 0, 0, 3 - 2*n],
            [n, n+2, -2, 0, 0, -2-n],
            [0, 2, -1, 3, 0, 6],
            [0, 0, 1+n, n, 1, -1+2*n],
            [0, 0, 0, -n, n+1, -3*n]])

M = M.subs(n, 5)
print('Исходная матрица: \n \\[ \n A=' + print_matrix(np.array(M), type_of_brace('p'), precision_value) + '\n \\]')


def progonka(matrix, order, idx):
    print('Прогонка, начиная с первой строки:\n')
    for i in range(idx):
        if (i != 0):
            matrix = matrix.elementary_row_op(op="n->n+km",k=-Rational(matrix[i,i-1], matrix[i-1,i-1]),row2=i-1,row1=i)
            print('\\[ \n' + print_matrix(np.array(matrix), type_of_brace('p'), precision_value) + '\n \\rightarrow \n \\]')
    print('Прогонка, начиная с последней строки:\n')
    for j in reversed(range(idx-1, order)):
        if (j != idx-1):
            matrix = matrix.elementary_row_op(op="n->n+km",k=-Rational(matrix[j-1,j], matrix[j,j]),row2=j,row1=j-1)
            print('\\[ \n' + print_matrix(np.array(matrix), type_of_brace('p'), precision_value) + '\n \\rightarrow \n \\]')
    return matrix, matrix[idx-1, order] / matrix[idx-1,idx-1]


M, sol = progonka(M,5,2)
print('Таким образом, получаем, что $x_{2} = ' + str(sol) + '$.')

Исходная матрица: 
 \[ 
 A=\begin{pmatrix}
     1 & 4 & 0 & 0 & 0 & -7 \\
     5 & 7 & -2 & 0 & 0 & -7 \\
     0 & 2 & -1 & 3 & 0 & 6 \\
     0 & 0 & 6 & 5 & 1 & 9 \\
     0 & 0 & 0 & -5 & 6 & -15 
  \end{pmatrix}
 \]
Прогонка, начиная с первой строки:

\[ 
\begin{pmatrix}
     1 & 4 & 0 & 0 & 0 & -7 \\
     0 & -13 & -2 & 0 & 0 & 28 \\
     0 & 2 & -1 & 3 & 0 & 6 \\
     0 & 0 & 6 & 5 & 1 & 9 \\
     0 & 0 & 0 & -5 & 6 & -15 
  \end{pmatrix}
 \rightarrow 
 \]
Прогонка, начиная с последней строки:

\[ 
\begin{pmatrix}
     1 & 4 & 0 & 0 & 0 & -7 \\
     0 & -13 & -2 & 0 & 0 & 28 \\
     0 & 2 & -1 & 3 & 0 & 6 \\
     0 & 0 & 6 & 35/6 & 0 & 23/2 \\
     0 & 0 & 0 & -5 & 6 & -15 
  \end{pmatrix}
 \rightarrow 
 \]
\[ 
\begin{pmatrix}
     1 & 4 & 0 & 0 & 0 & -7 \\
     0 & -13 & -2 & 0 & 0 & 28 \\
     0 & 2 & -143/35 & 0 & 0 & 3/35 \\
     0 & 0 & 6 & 35/6 & 0 & 23/2 \\
     0 & 0 & 0 & -5 & 6 & -15 
  \end{pmatrix}
 \rightarrow 
 \]
\[ 
\begin{pmatrix}
     1 & 4 & 0 & 0 & 0 & -7 \\
   

# Метод вращений

In [107]:
n = symbols('n')

M = Matrix([[4*n, -12, 10, -12+2*n],
            [-3*n, 9, -5, 9+n],
            [12*n, 3, 3, 3-21*n]])

M = M.subs(n, 5)

M = M.subs(n, 5)
print('Исходная матрица: \n \\[ \n A=' + print_matrix(np.array(M), type_of_brace('p'), precision_value) + '\n \\]')


def vrashenya(matrix):
    invQ = eye(3)
    v_list = []
    print('Преобразования исходной матрицы:')
    print('\\[')
    for j in range(3):
        for i in range(j+1,3):
            cc = matrix[j,j] / sqrt(matrix[j,j]**2 + matrix[i,j]**2) # cos
            ss = - matrix[i,j] / sqrt(matrix[j,j]**2 + matrix[i,j]**2) # sin
            v = eye(3)
            v[j,j] = cc
            v[i,i] = cc
            v[j,i] = ss
            v[i,j] = -ss
            v_list.append((v, i, j))
            for u in range(4):
                a = matrix[j,u]
                b = matrix[i,u]
                matrix[j,u] = a*cc - b*ss
                matrix[i,u] = a*ss + b*cc
            for u in range(3):
                qa = invQ[j,u]
                qb = invQ[i,u]
                invQ[j,u] = qa*cc - qb*ss
                invQ[i,u] = qa*ss + qb*cc
            print(print_matrix(np.array(matrix), type_of_brace('p'), precision_value) + '\n \\rightarrow')
    print('\\]')
    print('Где матрицы вращения имеют следующий вид:')
    print('\\[')
    for v in v_list:
        print('  V_{' + str(v[1]) + str(v[2]) + '} =' + print_matrix(np.array(v[0]), type_of_brace('p'), precision_value) + ', \\;\\;\\;')   
    print('\\]')
    return matrix, invQ.transpose()

M, Q = vrashenya(M)
b = M[0:3,0:3]
print('Таким образом, матрица $Q$ имеет следующий вид:')
print('\\[ \n Q = V_{10}^T \cdot V_{20}^T \cdot V_{21}^T = ')
print(print_matrix(np.array(Q), type_of_brace('p'), precision_value) + '\n \\]')

def solveUxy(matrix, y):
    solution = zeros(3,1)
    for i in reversed(range(3)):
        curr_sol = Rational(y[i], matrix[i,i])
        for j in range(i):
            y[j] = y[j] - y[i] * Rational(matrix[j,i], matrix[i,i])
        solution[i] = curr_sol
    return solution

print(f'Решая последовательно системы $Qx_1 = (-2, 14, -102)^T$ и $Rx = x_1$, получаем ответ $x = {tuple(solveUxy(b, M[0:3,3]))}$.' )


Исходная матрица: 
 \[ 
 A=\begin{pmatrix}
     20 & -12 & 10 & -2 \\
     -15 & 9 & -5 & 14 \\
     60 & 3 & 3 & -102 
  \end{pmatrix}
 \]
Преобразования исходной матрицы:
\[
\begin{pmatrix}
     25 & -15 & 11 & -10 \\
     0 & 0 & 2 & 10 \\
     60 & 3 & 3 & -102 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     65 & -3 & 7 & -98 \\
     0 & 0 & 2 & 10 \\
     0 & 15 & -9 & -30 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     65 & -3 & 7 & -98 \\
     0 & 15 & -9 & -30 \\
     0 & 0 & -2 & -10 
  \end{pmatrix}
 \rightarrow
\]
Где матрицы вращения имеют следующий вид:
\[
  V_{10} =\begin{pmatrix}
     4/5 & 3/5 & 0 \\
     -3/5 & 4/5 & 0 \\
     0 & 0 & 1 
  \end{pmatrix}, \;\;\;
  V_{20} =\begin{pmatrix}
     5/13 & 0 & -12/13 \\
     0 & 1 & 0 \\
     12/13 & 0 & 5/13 
  \end{pmatrix}, \;\;\;
  V_{21} =\begin{pmatrix}
     1 & 0 & 0 \\
     0 & 0 & -1 \\
     0 & 1 & 0 
  \end{pmatrix}, \;\;\;
\]
Таким образом, матрица $Q$ имеет следующий вид:
\[ 
 Q = V_{10}^T \cdot V_{20}^T \cd

# Метод отражений

In [128]:
n = symbols('n')

M = Matrix([[3*n+3, 12*n, 2*n, 7*n-n**2],
            [-6*n-6,-4*n, n, n+7*n**2],
            [6*n+6,2*n,0,-4*n-6*n**2]])

M = M.subs(n, 5)
b_old = M[:, 3]
print('Исходная матрица для $n = 5$: \n \\[ \n A=' + print_matrix(np.array(M), type_of_brace('p'), precision_value) + '\n \\]')

def otrazhenya(matrix):
    invQ = eye(3)
    v_list = []
    print('Преобразования исходной матрицы:')
    print('\\[')
    for i in range(3):
        a = matrix[i:3,i]
        ash = zeros(3-i,1)
        ash[0,0] = -sign(a[0,0]) * sqrt(a.dot(a))
        w = (a-ash) / sqrt((a-ash).dot(a-ash))
        w_small = w * w.transpose()
        w_big = zeros(3)
        for a in range(np.array(w_small).shape[0]):
            for b in range(np.array(w_small).shape[1]):
                w_big[a + i, b + i] = w_small[a, b]
        W_m = eye(3) - 2 * w_big
        v_list.append((W_m, i))
        for j in range(4):
            matrix[i:3,j] = matrix[i:3,j] - 2*(matrix[i:3,j].dot(w))*w
        for j in range(3):
            invQ[i:3,j] = invQ[i:3,j] - 2*(invQ[i:3,j].dot(w))*w  
        print(print_matrix(np.array(matrix), type_of_brace('p'), precision_value) + '\n \\rightarrow')
    print('\\]')
    print('Где матрицы отражения имеют следующий вид:')
    print('\\[')
    for v in v_list:
        print('  V_{' + str(v[1]) + '} =' + print_matrix(np.array(v[0]), type_of_brace('p'), precision_value) + ', \\;\\;\\;')   
    print('\\]')
    return matrix, invQ.transpose()

M, Q = otrazhenya(M)
b = M[0:3, 0:3]

print('Таким образом, матрица $Q$ имеет следующий вид:')
print('\\[ \n Q = V_{0}^T \cdot V_{1}^T \cdot V_{2}^T = ')
print(print_matrix(np.array(Q), type_of_brace('p'), precision_value) + '\n \\]')

def solveUxy(matrix, y):
    solution = zeros(3,1)
    for i in reversed(range(3)):
        curr_sol = Rational(y[i], matrix[i,i])
        for j in range(i):
            y[j] = y[j] - y[i] * Rational(matrix[j,i], matrix[i,i])
        solution[i] = curr_sol
    return solution

print(f'Решая последовательно системы $Qx_1 = {tuple(b_old)}^T$ и $Rx = x_1$, получаем ответ $x = {tuple(solveUxy(b, M[0:3,3]))}$.' )

Исходная матрица для $n = 5$: 
 \[ 
 A=\begin{pmatrix}
     18 & 60 & 10 & 10 \\
     -36 & -20 & 5 & 180 \\
     36 & 10 & 0 & -170 
  \end{pmatrix}
 \]
Преобразования исходной матрицы:
\[
\begin{pmatrix}
     -54 & -40 & 0 & 230 \\
     0 & 30 & 10 & 70 \\
     0 & -40 & -5 & -60 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     -54 & -40 & 0 & 230 \\
     0 & -50 & -10 & -90 \\
     0 & 0 & 5 & 20 
  \end{pmatrix}
 \rightarrow
\begin{pmatrix}
     -54 & -40 & 0 & 230 \\
     0 & -50 & -10 & -90 \\
     0 & 0 & -5 & -20 
  \end{pmatrix}
 \rightarrow
\]
Где матрицы отражения имеют следующий вид:
\[
  V_{0} =\begin{pmatrix}
     -1/3 & 2/3 & -2/3 \\
     2/3 & 2/3 & 1/3 \\
     -2/3 & 1/3 & 2/3 
  \end{pmatrix}, \;\;\;
  V_{1} =\begin{pmatrix}
     1 & 0 & 0 \\
     0 & -3/5 & 4/5 \\
     0 & 4/5 & 3/5 
  \end{pmatrix}, \;\;\;
  V_{2} =\begin{pmatrix}
     1 & 0 & 0 \\
     0 & 1 & 0 \\
     0 & 0 & -1 
  \end{pmatrix}, \;\;\;
\]
Таким образом, матрица $Q$ имеет следующий вид:
\[ 
 Q 